In [ ]:
import numpy as np
import pandas as pd
books = pd.read_csv("BX-Books.csv", sep=';', encoding="latin-1", error_bad_lines=False)
users = pd.read_csv("BX-Users.csv", sep=';', encoding="latin-1", error_bad_lines=False)
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=';', encoding="latin-1", error_bad_lines=False)

In [ ]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
users.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [ ]:

x = ratings['user_id'].value_counts() > 200
y = x[x].index  #user_ids
print(y.shape)
ratings = ratings[ratings['user_id'].isin(y)]
ratings

In [ ]:
rating_with_books = ratings.merge(books, on='ISBN')
rating_with_books.head()

In [ ]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating

In [ ]:
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)

In [ ]:
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating

In [ ]:
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating

In [ ]:
book_pivot = final_rating.pivot_table(columns= 'user_id', index='title', values = 'rating')

In [ ]:
book_pivot.fillna(0, inplace = True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')
model.fit(book_sparse)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[100, :].values.reshape(1,-1))
suggestions

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])